

## <p style="text-align: center;">Project 3 - Machine Learning using Tensorflow and AWS</p>


Your project should be written in a **Jupyter notebook**. You may work in groups of two if you wish. Only one student per team needs to submit the assignment on Canvas.  But be sure to include name and UTEID for both students.

Also, please make sure your code runs and the graphics (and anything else) are displayed in your notebook before submitting. (%matplotlib inline)

This project is on Tensorflow, its usage, Cloud services and why they are helpful in analyzing Big Data.

In this Project, we are trying to analyze crime data. This data is taken from <a href=https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-present/ijzp-q8t2/data> here </a>. You have to download data from here, it should be around 1.6 GB in size. 

You have to predict type of crime that can happen given the features.

You have to complete all the four tasks and report your observations for each if any. There is a bonus task as well, it has two options and you can choose any one of it.

# Task 1 - Read the data and filter the data you need

This is open ended, you can use any features you want and drop any one of them. The 'Primary Type' is our target variable which we will be predicting for.
Split the data into training and testing in the ratio of 80:20.

Before starting our analysis, we import some necessary packages.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time

# setting display parameters
%matplotlib inline
pd.set_option('display.max_columns', None)

We first import the data in from the downloaded csv file into a pandas dataframe.

In [38]:
t0 = time.clock()
crime_data = pd.read_csv('crimes.csv')

t1 = time.clock()
total = t1-t0
print("Time to read crimes data = "+str(total) +" seconds")

Time to read crimes data = 30.829999999999984 seconds


In [15]:
crime_data.columns = [c.replace(' ', '_') for c in crime_data.columns]

In [4]:
crime_data.head()

,ID,Case_Number,Date,Block,IUCR,Primary_Type,Description,Location_Description,Arrest,Domestic,Beat,District,Ward,Community_Area,FBI_Code,X_Coordinate,Y_Coordinate,Year,Updated_On,Latitude,Longitude,Location
0,10000092,HY189866,03/18/2015 07:44:00 PM,047XX W OHIO ST,041A,BATTERY,AGGRAVATED: HANDGUN,STREET,False,False,1111,11.0,28.0,25.0,04B,1144606.0,1903566.0,2015,02/10/2018 03:50:01 PM,41.891399,-87.744385,"(41.891398861, -87.744384567)"
1,10000094,HY190059,03/18/2015 11:00:00 PM,066XX S MARSHFIELD AVE,4625,OTHER OFFENSE,PAROLE VIOLATION,STREET,True,False,725,7.0,15.0,67.0,26,1166468.0,1860715.0,2015,02/10/2018 03:50:01 PM,41.773372,-87.665319,"(41.773371528, -87.665319468)"
2,10000095,HY190052,03/18/2015 10:45:00 PM,044XX S LAKE PARK AVE,0486,BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,False,True,222,2.0,4.0,39.0,08B,1185075.0,1875622.0,2015,02/10/2018 03:50:01 PM,41.813861,-87.596643,"(41.81386068, -87.596642837)"
3,10000096,HY190054,03/18/2015 10:30:00 PM,051XX S MICHIGAN AVE,0460,BATTERY,SIMPLE,APARTMENT,False,False,225,2.0,3.0,40.0,08B,1178033.0,1870804.0,2015,02/10/2018 03:50:01 PM,41.800802,-87.622619,"(41.800802415, -87.622619343)"
4,10000097,HY189976,03/18/2015 09:00:00 PM,047XX W ADAMS ST,031A,ROBBERY,ARMED: HANDGUN,SIDEWALK,False,False,1113,11.0,28.0,25.0,03,1144920.0,1898709.0,2015,02/10/2018 03:50:01 PM,41.878065,-87.743354,"(41.878064761, -87.743354013)"


Looking at the features, we can see that there are columns that do not add any information. In specific:
1. ID
2. Case Number
3. Updated On
4. Location (we have separate features for Lat and Long)

As a result, we can remove the above, without reducing the accuracy of our model later on. In addition, the "Year" feature is already contained in the "Date" feature, so we will probably remove that as well at a later stage.

In [5]:
crime_data.shape

(6720963, 22)

In [6]:
crime_data = crime_data.drop(['ID'], axis=1)
crime_data = crime_data.drop(['Case_Number'], axis=1)
crime_data = crime_data.drop(['Updated_On'], axis=1)
crime_data = crime_data.drop(['Location'], axis=1)

In [7]:
crime_data.shape

(6720963, 18)

In [8]:
crime_data.head()

,Date,Block,IUCR,Primary_Type,Description,Location_Description,Arrest,Domestic,Beat,District,Ward,Community_Area,FBI_Code,X_Coordinate,Y_Coordinate,Year,Latitude,Longitude
0,03/18/2015 07:44:00 PM,047XX W OHIO ST,041A,BATTERY,AGGRAVATED: HANDGUN,STREET,False,False,1111,11.0,28.0,25.0,04B,1144606.0,1903566.0,2015,41.891399,-87.744385
1,03/18/2015 11:00:00 PM,066XX S MARSHFIELD AVE,4625,OTHER OFFENSE,PAROLE VIOLATION,STREET,True,False,725,7.0,15.0,67.0,26,1166468.0,1860715.0,2015,41.773372,-87.665319
2,03/18/2015 10:45:00 PM,044XX S LAKE PARK AVE,0486,BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,False,True,222,2.0,4.0,39.0,08B,1185075.0,1875622.0,2015,41.813861,-87.596643
3,03/18/2015 10:30:00 PM,051XX S MICHIGAN AVE,0460,BATTERY,SIMPLE,APARTMENT,False,False,225,2.0,3.0,40.0,08B,1178033.0,1870804.0,2015,41.800802,-87.622619
4,03/18/2015 09:00:00 PM,047XX W ADAMS ST,031A,ROBBERY,ARMED: HANDGUN,SIDEWALK,False,False,1113,11.0,28.0,25.0,03,1144920.0,1898709.0,2015,41.878065,-87.743354


Further to the below, we will break the "Date" feature into the following separate features:
1. Month
2. Day
3. Time

Due to the size of the data, we will need to break it down into smaller parts.

In [15]:
t0 = time.clock()

broken_date1_9 = [(int(crime_data.iloc[:]['Date'][i][:2]),
                  int(crime_data.iloc[:]['Date'][i][3:5]),
                  int(crime_data.iloc[:]['Date'][i][11:13]+crime_data.iloc[:]['Date'][i][14:16]),
                  crime_data.iloc[:]['Date'][i][20:22])
                  for i in range(746774)]

t1 = time.clock()
total = t1-t0
print("Time to run = "+str(total) +" seconds")

Time to run = 487.29229300000003 seconds


In [20]:
t0 = time.clock()

broken_date2_9 = [(int(crime_data.iloc[:]['Date'][i][:2]),
                  int(crime_data.iloc[:]['Date'][i][3:5]),
                  int(crime_data.iloc[:]['Date'][i][11:13]+crime_data.iloc[:]['Date'][i][14:16]),
                  crime_data.iloc[:]['Date'][i][20:22])
                  for i in range(746774,1493548)]

t1 = time.clock()
total = t1-t0
print("Time to run = "+str(total) +" seconds")

Time to run = 499.82779200000004 seconds


In [22]:
t0 = time.clock()

broken_date3_9 = [(int(crime_data.iloc[:]['Date'][i][:2]),
                  int(crime_data.iloc[:]['Date'][i][3:5]),
                  int(crime_data.iloc[:]['Date'][i][11:13]+crime_data.iloc[:]['Date'][i][14:16]),
                  crime_data.iloc[:]['Date'][i][20:22])
                  for i in range(1493548,2240321)]

t1 = time.clock()
total = t1-t0
print("Time to run = "+str(total) +" seconds")

Time to run = 492.52064900000005 seconds


In [24]:
t0 = time.clock()

broken_date4_9 = [(int(crime_data.iloc[:]['Date'][i][:2]),
                  int(crime_data.iloc[:]['Date'][i][3:5]),
                  int(crime_data.iloc[:]['Date'][i][11:13]+crime_data.iloc[:]['Date'][i][14:16]),
                  crime_data.iloc[:]['Date'][i][20:22])
                  for i in range(2240321,2987095)]

t1 = time.clock()
total = t1-t0
print("Time to run = "+str(total) +" seconds")

Time to run = 498.86600299999986 seconds


In [26]:
t0 = time.clock()

broken_date5_9 = [(int(crime_data.iloc[:]['Date'][i][:2]),
                  int(crime_data.iloc[:]['Date'][i][3:5]),
                  int(crime_data.iloc[:]['Date'][i][11:13]+crime_data.iloc[:]['Date'][i][14:16]),
                  crime_data.iloc[:]['Date'][i][20:22])
                  for i in range(2987095,3733869)]

t1 = time.clock()
total = t1-t0
print("Time to run = "+str(total) +" seconds")

Time to run = 495.15978299999983 seconds


In [28]:
t0 = time.clock()

broken_date6_9 = [(int(crime_data.iloc[:]['Date'][i][:2]),
                  int(crime_data.iloc[:]['Date'][i][3:5]),
                  int(crime_data.iloc[:]['Date'][i][11:13]+crime_data.iloc[:]['Date'][i][14:16]),
                  crime_data.iloc[:]['Date'][i][20:22])
                  for i in range(3733869,4480643)]

t1 = time.clock()
total = t1-t0
print("Time to run = "+str(total) +" seconds")

Time to run = 505.31637699999965 seconds


In [32]:
t0 = time.clock()

broken_date7_9 = [(int(crime_data.iloc[:]['Date'][i][:2]),
                  int(crime_data.iloc[:]['Date'][i][3:5]),
                  int(crime_data.iloc[:]['Date'][i][11:13]+crime_data.iloc[:]['Date'][i][14:16]),
                  crime_data.iloc[:]['Date'][i][20:22])
                  for i in range(4480643,5227417)]

t1 = time.clock()
total = t1-t0
print("Time to run = "+str(total) +" seconds")

Time to run = 507.7619129999998 seconds


In [33]:
t0 = time.clock()

broken_date8_9 = [(int(crime_data.iloc[:]['Date'][i][:2]),
                  int(crime_data.iloc[:]['Date'][i][3:5]),
                  int(crime_data.iloc[:]['Date'][i][11:13]+crime_data.iloc[:]['Date'][i][14:16]),
                  crime_data.iloc[:]['Date'][i][20:22])
                  for i in range(5227417,5974191)]

t1 = time.clock()
total = t1-t0
print("Time to run = "+str(total) +" seconds")

Time to run = 499.0614609999998 seconds


In [34]:
t0 = time.clock()

broken_date9_9 = [(int(crime_data.iloc[:]['Date'][i][:2]),
                  int(crime_data.iloc[:]['Date'][i][3:5]),
                  int(crime_data.iloc[:]['Date'][i][11:13]+crime_data.iloc[:]['Date'][i][14:16]),
                  crime_data.iloc[:]['Date'][i][20:22])
                  for i in range(5974191,6720963)]

t1 = time.clock()
total = t1-t0
print("Time to run = "+str(total) +" seconds")

Time to run = 494.94930099999965 seconds


In [83]:
list_dates = broken_date1_9 + broken_date2_9 + broken_date3_9 + \
             broken_date4_9 + broken_date5_9 + broken_date6_9 + \
             broken_date7_9 + broken_date8_9 + broken_date9_9

In [94]:
list_dates_pm = []

for i in range(6720963):
    a = list_dates[i][0]
    b = list_dates[i][1]
    if list_dates[i][3] == 'PM':
        c = list_dates[i][2] + 1200
    else:
        c = list_dates[i][2]
    list_dates_pm.append((a,b,c))

In [103]:
broken_dates = pd.DataFrame(list_dates_pm)
broken_dates.rename(columns={0: 'month', 1: 'day', 2: 'time', 3: 'ampm'}, inplace=True)

In [102]:
broken_dates.shape

(6720963, 3)

In [104]:
broken_dates.head()

,month,day,time
0,3,18,1944
1,3,18,2300
2,3,18,2245
3,3,18,2230
4,3,18,2100


In [105]:
crime_data.shape

(6720963, 18)

In [120]:
crime_data.insert(0, 'month', np.array([i[0] for i in list_dates_pm]))
crime_data.insert(0, 'day', np.array([i[1] for i in list_dates_pm]))
crime_data.insert(0, 'time', np.array([i[2] for i in list_dates_pm]))

crime_data = crime_data.drop(['Date'], axis=1)

In [121]:
crime_data.shape

(6720963, 20)

In [122]:
crime_data.head()

,time,day,month,Block,IUCR,Primary_Type,Description,Location_Description,Arrest,Domestic,Beat,District,Ward,Community_Area,FBI_Code,X_Coordinate,Y_Coordinate,Year,Latitude,Longitude
0,1944,18,3,047XX W OHIO ST,041A,BATTERY,AGGRAVATED: HANDGUN,STREET,False,False,1111,11.0,28.0,25.0,04B,1144606.0,1903566.0,2015,41.891399,-87.744385
1,2300,18,3,066XX S MARSHFIELD AVE,4625,OTHER OFFENSE,PAROLE VIOLATION,STREET,True,False,725,7.0,15.0,67.0,26,1166468.0,1860715.0,2015,41.773372,-87.665319
2,2245,18,3,044XX S LAKE PARK AVE,0486,BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,False,True,222,2.0,4.0,39.0,08B,1185075.0,1875622.0,2015,41.813861,-87.596643
3,2230,18,3,051XX S MICHIGAN AVE,0460,BATTERY,SIMPLE,APARTMENT,False,False,225,2.0,3.0,40.0,08B,1178033.0,1870804.0,2015,41.800802,-87.622619
4,2100,18,3,047XX W ADAMS ST,031A,ROBBERY,ARMED: HANDGUN,SIDEWALK,False,False,1113,11.0,28.0,25.0,03,1144920.0,1898709.0,2015,41.878065,-87.743354


In [123]:
crime_data.describe()

,time,day,month,Beat,District,Ward,Community_Area,X_Coordinate,Y_Coordinate,Year,Latitude,Longitude
count,6.720963e+06,6.720963e+06,6.720963e+06,6.720963e+06,6.720916e+06,6.106110e+06,6.104935e+06,6.660776e+06,6.660776e+06,6.720963e+06,6.660776e+06,6.660776e+06
mean,1.474307e+03,1.562430e+01,6.521557e+00,1.192123e+03,1.130310e+01,2.267172e+01,3.758912e+01,1.164513e+06,1.885705e+06,2.008282e+03,4.184197e+01,-8.767182e+01
std,6.377037e+02,8.834725e+00,3.328148e+00,7.034665e+02,6.946155e+00,1.383100e+01,2.153240e+01,1.718000e+04,3.272399e+04,4.980901e+00,9.004859e-02,6.216797e-02
min,1.000000e+02,1.000000e+00,1.000000e+00,1.110000e+02,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.001000e+03,3.661945e+01,-9.168657e+01
25%,1.030000e+03,8.000000e+00,4.000000e+00,6.220000e+02,6.000000e+00,1.000000e+01,2.300000e+01,1.152933e+06,1.859179e+06,2.004000e+03,4.176889e+01,-8.771388e+01
50%,1.530000e+03,1.600000e+01,7.000000e+00,1.111000e+03,1.000000e+01,2.200000e+01,3.200000e+01,1.165982e+06,1.890530e+06,2.008000e+03,4.185518e+01,-8.766620e+01
75%,2.005000e+03,2.300000e+01,9.000000e+00,1.731000e+03,1.700000e+01,3.400000e+01,5.800000e+01,1.176352e+06,1.909315e+06,2.012000e+03,4.190687e+01,-8.762839e+01
max,2.459000e+03,3.100000e+01,1.200000e+01,2.535000e+03,3.100000e+01,5.000000e+01,7.700000e+01,1.205119e+06,1.951622e+06,2.018000e+03,4.202291e+01,-8.752453e+01


Analysing further on our data, we can see that the following features are categorical:
1. Block
2. IUCR (Illinois Uniform Crime Reporting)
3. Description
4. Location
5. Arrest
6. Domestic
7. Beat
8. District
9. Ward
10. Community Area
11. FBI Code

As a result making sure that the type of the data is in the right format, we will generate dummy variables, implementing one hot encoding.

In [2]:
t0 = time.clock()

crime_data = pd.read_csv('broken_date.csv')

t1 = time.clock()
total = t1-t0
print("Time to read the broken file = "+str(total) +" seconds")

Time to read the broken file = 20.419999999999998 seconds


In [3]:
crime_data['Block'] = crime_data.Block.astype('category')
crime_data['IUCR'] = crime_data.IUCR.astype('category')
crime_data['Description'] = crime_data.Description.astype('category')
crime_data['Location_Description'] = crime_data.Location_Description.astype('category')
crime_data['Arrest'] = crime_data.Arrest.astype('category')
crime_data['Domestic'] = crime_data.Domestic.astype('category')
crime_data['Beat'] = crime_data.Beat.astype('category')
crime_data['District'] = crime_data.District.astype('category')
crime_data['Ward'] = crime_data.Ward.astype('category')
crime_data['Community_Area'] = crime_data.Community_Area.astype('category')
crime_data['FBI_Code'] = crime_data.FBI_Code.astype('category')

crime_data = crime_data.drop(['Unnamed: 0'], axis=1)

Due to the sparcity of the data, we decided to remove the "Block" and "IUCR" columns since we already have location information.

In [4]:
crime_data = crime_data.drop(['Block'], axis=1)
crime_data = crime_data.drop(['IUCR'], axis=1)

Next we will drop all the rows with NaN values, in order to ensure accuracy in our predictions.

In [5]:
crime_data.shape

(6720963, 18)

In [6]:
crime_data = crime_data.dropna()

In [7]:
crime_data.shape

(6051093, 18)

We separate the "Primary_type" data which is the outcome and we don't want it to be broken down into dummy_variables.

In [8]:
primary_type = crime_data.loc[:,'Primary_Type']
crime_data = crime_data.drop(['Primary_Type'], axis=1)

In [9]:
uniq = primary_type.nunique()

In [10]:
uniq # There are 34 unique values for the Y label

34

In [11]:
crime_data.head()

,time,day,month,Description,Location_Description,Arrest,Domestic,Beat,District,Ward,Community_Area,FBI_Code,X_Coordinate,Y_Coordinate,Year,Latitude,Longitude
0,1944,18,3,AGGRAVATED: HANDGUN,STREET,False,False,1111,11.0,28.0,25.0,04B,1144606.0,1903566.0,2015,41.891399,-87.744385
1,2300,18,3,PAROLE VIOLATION,STREET,True,False,725,7.0,15.0,67.0,26,1166468.0,1860715.0,2015,41.773372,-87.665319
2,2245,18,3,DOMESTIC BATTERY SIMPLE,APARTMENT,False,True,222,2.0,4.0,39.0,08B,1185075.0,1875622.0,2015,41.813861,-87.596643
3,2230,18,3,SIMPLE,APARTMENT,False,False,225,2.0,3.0,40.0,08B,1178033.0,1870804.0,2015,41.800802,-87.622619
4,2100,18,3,ARMED: HANDGUN,SIDEWALK,False,False,1113,11.0,28.0,25.0,03,1144920.0,1898709.0,2015,41.878065,-87.743354


In [13]:
t0 = time.clock()
crime_data = pd.get_dummies(crime_data)
t1 = time.clock()
total = t1-t0
print("Time to run = "+str(total) +" seconds")

Time to run = 66.84 seconds


In [14]:
crime_data.shape

(6051093, 1052)

In [120]:
crime_data.head()

,time,day,month,X_Coordinate,Y_Coordinate,Year,Latitude,Longitude,Description_$300 AND UNDER,Description_$500 AND UNDER,Description_ABUSE/NEGLECT: CARE FACILITY,Description_ADULTRY,Description_AGG CRIM SEX ABUSE FAM MEMBER,Description_AGG CRIMINAL SEXUAL ABUSE,Description_AGG PO HANDS ETC SERIOUS INJ,Description_AGG PO HANDS NO/MIN INJURY,Description_AGG PRO EMP HANDS SERIOUS INJ,Description_AGG PRO.EMP: HANDGUN,Description_AGG PRO.EMP: OTHER DANG WEAPON,Description_AGG PRO.EMP: OTHER FIREARM,Description_AGG PRO.EMP:KNIFE/CUTTING INST,Description_AGG RIT MUT: HANDS/FIST/FEET NO/MINOR INJURY,Description_AGG RIT MUT: HANDS/FIST/FEET SERIOUS INJURY,Description_AGG RITUAL MUT:HANDGUN,Description_AGG RITUAL MUT:KNIFE/CUTTING I,Description_AGG RITUAL MUT:OTH DANG WEAPON,Description_AGG SEX ASSLT OF CHILD FAM MBR,Description_AGG: FINANCIAL ID THEFT,Description_AGG: HANDS/FIST/FEET NO/MINOR INJURY,Description_AGG: HANDS/FIST/FEET SERIOUS INJURY,Description_AGGRAVATED,Description_AGGRAVATED COMPUTER TAMPERING,Description_AGGRAVATED DOMESTIC BATTERY,Description_AGGRAVATED DOMESTIC BATTERY: HANDGUN,Description_AGGRAVATED DOMESTIC BATTERY: HANDS/FIST/FEET SERIOUS INJURY,Description_AGGRAVATED DOMESTIC BATTERY: KNIFE/CUTTING INST,Description_AGGRAVATED DOMESTIC BATTERY: OTHER DANG WEAPON,Description_AGGRAVATED DOMESTIC BATTERY: OTHER FIREARM,Description_AGGRAVATED FINANCIAL IDENTITY THEFT,Description_AGGRAVATED OF A CHILD,Description_AGGRAVATED OF A SENIOR CITIZEN,Description_AGGRAVATED OF A UNBORN CHILD,Description_AGGRAVATED PO: HANDGUN,Description_AGGRAVATED PO: KNIFE/CUT INSTR,Description_AGGRAVATED PO: OTHER DANG WEAP,Description_AGGRAVATED PO: OTHER FIREARM,Description_AGGRAVATED PO:KNIFE/CUT INSTR,Description_AGGRAVATED VEHICULAR HIJACKING,Description_AGGRAVATED: HANDGUN,Description_AGGRAVATED: KNIFE/CUT INSTR,Description_AGGRAVATED: OTHER,Description_AGGRAVATED: OTHER DANG WEAPON,Description_AGGRAVATED: OTHER FIREARM,Description_AGGRAVATED:KNIFE/CUTTING INSTR,Description_AIDING ARRESTEE ESCAPE,Description_ALTER COINS,Description_ALTER/FORGE PRESCRIPTION,Description_ANIMAL ABUSE/NEGLECT,Description_ANIMAL FIGHTING,Description_ARMED VIOLENCE,Description_ARMED WHILE UNDER THE INFLUENCE,Description_ARMED: HANDGUN,Description_ARMED: OTHER DANGEROUS WEAPON,Description_ARMED: OTHER FIREARM,Description_ARMED:KNIFE/CUTTING INSTRUMENT,Description_ARSON THREAT,Description_ARSONIST: ANNUAL REGISTRATION,Description_ARSONIST: DUTY TO REGISTER,Description_ARSONIST: FAIL TO REGISTER NEW ADDRESS,Description_ATT AGG CRIM SEXUAL ABUSE,Description_ATT AGG CRIMINAL SEXUAL ABUSE,Description_ATT CRIM SEXUAL ABUSE,Description_ATT: AUTOMOBILE,"Description_ATT: TRUCK, BUS, MOTOR HOME",Description_ATTEMPT - FINANCIAL IDENTITY THEFT,Description_ATTEMPT AGG: HANDGUN,Description_ATTEMPT AGG: KNIFE/CUT INSTR,Description_ATTEMPT AGG: OTHER,Description_ATTEMPT AGG: OTHER DANG WEAPON,Description_ATTEMPT AGG: OTHER FIREARM,Description_ATTEMPT ARSON,Description_ATTEMPT FINANCIAL IDENTITY THEFT,Description_ATTEMPT FORCIBLE ENTRY,Description_ATTEMPT NON-AGGRAVATED,Description_ATTEMPT POSSESSION CANNABIS,Description_ATTEMPT POSSESSION NARCOTICS,Description_ATTEMPT THEFT,Description_ATTEMPT: AGGRAVATED,Description_ATTEMPT: ARMED-HANDGUN,Description_ATTEMPT: ARMED-KNIFE/CUT INSTR,Description_ATTEMPT: ARMED-OTHER DANG WEAP,Description_ATTEMPT: ARMED-OTHER FIREARM,"Description_ATTEMPT: CYCLE, SCOOTER, BIKE NO VIN","Description_ATTEMPT: CYCLE, SCOOTER, BIKE W-VIN",Description_ATTEMPT: STRONGARM-NO WEAPON,Description_AUTOMOBILE,Description_BIGAMY,Description_BOARD PLANE WITH WEAPON,Description_BOGUS CHECK,Description_BOLITA OR BOLI PUL/OFFICE,Description_BOLITA OR BOLI PUL/RUNNER,Description_BOLITA OR BOLI PUL/WRITER,Description_BOMB THREAT,Description_BOOKMAKING/HORSES,Description_BOOKMAKING/SPORTS,Description_BRIBERY,Description_BY EXPLOSIVE,Description_BY FIRE,Description_CAB OPERATION,Description_CALCULATED CANNABIS CONSPIRACY,Description_CALL OPERATION,Description_CANNAB

In addition to the above, the featues (X Coordinate - Y Coordinate) and (Latitude, Longitude) are probably correlated, but due to lack of information at this point we will keep both and save the resulted data in a new file to be used for our modelling and further analysis.

In [12]:
# crime_data.to_csv('crime_data_new.csv')

After completion of the data preprocessing, we can now proceed with splitting them and creating training and testing sets to be used in our analysis.

# Task 2 - Convert the features in a form that can be given as input to tensorflow library/functions

Here you have to search a little bit. The data that we are using has data other than numerical values. Tensors take two kinds of values, either numeric or categorical. So, you have to convert the given data into a logical form which can be given to tensors.

<b>Due to the limitations of our AWS account, we decided to reduce the number of samples to avoid memory errors and ensure proper completion of the runnning code from 6.3 to 1 million samples.</b>

In [15]:
crime_data_reduced = crime_data.iloc[:1000000,:]
crime_data_reduced.shape

(1000000, 1052)

In [16]:
primary_type_reduced = primary_type[:1000000]
primary_type_reduced.shape

(1000000,)

Since the Y labels are multiclass & strings, we need to convert them into integer labels, and then flatten the Y vector into (i.e. perform one hot encoding on Y labels too).

In [17]:
ylabels,ylevels = pd.factorize(primary_type_reduced)

In [18]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder() 
lb = preprocessing.LabelBinarizer()

le.fit(ylevels.values)
y_encoded = le.transform(primary_type_reduced)

lb.fit(y_encoded)
Y_flat = lb.transform(y_encoded)

In [19]:
#Using scikit learn's train_test_split
from sklearn.model_selection import train_test_split

t0 = time.clock()
X = crime_data_reduced
y = Y_flat
# y = Y_flat[:,0]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

t1 = time.clock()
total = t1-t0
print("Time to split the data into train/test = "+str(total) +" seconds")

Time to split the data into train/test = 6.950000000000003 seconds


In [22]:
X_train.columns

Index(['time', 'day', 'month', 'X_Coordinate', 'Y_Coordinate', 'Year',
       'Latitude', 'Longitude', 'Description_$300 AND UNDER',
       'Description_$500 AND UNDER',
       ...
       'FBI_Code_14', 'FBI_Code_15', 'FBI_Code_16', 'FBI_Code_17',
       'FBI_Code_18', 'FBI_Code_19', 'FBI_Code_20', 'FBI_Code_22',
       'FBI_Code_24', 'FBI_Code_26'],
      dtype='object', length=1052)

After completion of our data preprocessing, we end up with 1,052 features and an index of size 34 for a total number of 1,000,000 samples.

# Task 3 - Build the model

Once the features and tensors are made, you have to build a machine learning model which, given the features predicts the type of crime. 
Report the accuracy of the model.

### We will be using logistic regression - Building the model

In [20]:
#convert pandas frames to numpy arrays for tensors to work on
X_train = X_train.values

In [21]:
X_test = X_test.values

In [34]:
y_train.shape

(800000, 34)

In [23]:
import tensorflow as tf
# Begin building the model framework
# Declare the variables that need to be learned and initialization
# There are 4 features here, A's dimension is (1052, 34)
A = tf.Variable(tf.random_normal(shape=[1052, 34]))
b = tf.Variable(tf.random_normal(shape=[34]))
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [24]:
# Define placeholders
data = tf.placeholder(dtype=tf.float32, shape=[None, 1052])
target = tf.placeholder(dtype=tf.float32, shape=[None, 34])

In [25]:
# Declare the model you need to learn
mod = tf.matmul(data, A) + b

In [26]:
# Declare loss function
# Use the sigmoid cross-entropy loss function,
# first doing a sigmoid on the model result and then using the cross-entropy loss function
loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=mod, labels=target))

In [27]:
# Define the learning rate， batch_size etc.
learning_rate = 0.003
batch_size = 50
iter_num = 10

In [28]:
# Define the optimizer
opt = tf.train.GradientDescentOptimizer(learning_rate)

In [29]:
# Define the goal
goal = opt.minimize(loss)

In [30]:
# Define the accuracy
# The default threshold is 0.5, rounded off directly
prediction = tf.round(tf.sigmoid(mod))
# Bool into float32 type
correct = tf.cast(tf.equal(prediction, target), dtype=tf.float32)
# Average
accuracy = tf.reduce_mean(correct)
# End of the definition of the model framework

In [31]:
# Start training model
# Define the variable that stores the result
loss_trace = []
train_acc = []
test_acc = []

In [33]:
t0 = time.clock()

# training model  
for epoch in range(iter_num):
    # Generate random batch index
    batch_index = np.random.choice(len(X_train), size=batch_size)
    #print(batch_index)
    batch_train_X = X_train[batch_index]
    batch_train_y = y_train[batch_index]
    
    print("for iter = ",epoch+1)
    
    #train the data for the chosen batch & also time it
    print("  starting to train model . . .")
    
    sess.run(goal, feed_dict={data: batch_train_X, target: batch_train_y})
    
    temp_loss = sess.run(loss, feed_dict={data: batch_train_X, target: batch_train_y})
    # convert into a matrix, and the shape of the placeholder to correspond
    temp_train_acc = sess.run(accuracy, feed_dict={data: X_train, target: y_train})
    temp_test_acc = sess.run(accuracy, feed_dict={data: X_test, target: y_test})
    # recode the result
    loss_trace.append(temp_loss)
    train_acc.append(temp_train_acc)
    test_acc.append(temp_test_acc)
    
    print("  temp_loss : ",temp_loss)
    print("  temp_train_acc : ",temp_train_acc)
    print("  temp_test_acc :",temp_test_acc)
    # output
    #if (epoch + 1) % 300 == 0:
print('epoch: {:4d} loss: {:5f} train_acc: {:5f} test_acc: {:5f}'.format(epoch + 1, temp_loss,temp_train_acc, temp_test_acc))


t1 = time.clock()
total = t1-t0
print("Total time taken to train this model "+str(total) +" seconds")

for iter =  1
  starting to train model . . .
  temp_loss :  4807813.0
  temp_train_acc :  0.9524442
  temp_test_acc : 0.95242953
for iter =  2
  starting to train model . . .
  temp_loss :  7603651.0
  temp_train_acc :  0.9545656
  temp_test_acc : 0.9544824
for iter =  3
  starting to train model . . .
  temp_loss :  6940857.5
  temp_train_acc :  0.9159506
  temp_test_acc : 0.91601676
for iter =  4
  starting to train model . . .
  temp_loss :  4034129.5
  temp_train_acc :  0.9412401
  temp_test_acc : 0.94124144
for iter =  5
  starting to train model . . .
  temp_loss :  2815942.2
  temp_train_acc :  0.9705767
  temp_test_acc : 0.9705775
for iter =  6
  starting to train model . . .
  temp_loss :  4926596.0
  temp_train_acc :  0.93643355
  temp_test_acc : 0.9363347
for iter =  7
  starting to train model . . .
  temp_loss :  6475169.0
  temp_train_acc :  0.9185038
  temp_test_acc : 0.91856354
for iter =  8
  starting to train model . . .
  temp_loss :  6755749.5
  temp_train_acc :  0

<b>From the above model training:  </b><br>
<b>Training Accuracy : 97.0577% </b><br>
<b> Testing Accuracy : 97.0570%</b><br>
<br>
<b>Time taken to train the model : 164.709 seconds</b>

# Task 4 - Run the model on your laptop and AWS cloud and observe the time taken for reading the data, training and testing. 

Report your observations. Due to more RAM needed for the same, it might be the case that it doesn't run on your laptop, so in that case report your laptop configuration and state it didnt run.

<p>Because the data size was huge (~1.6GB) , reading the data file is an extremely memory heavy task.</p>
<p>After reading and cleaning the data, performing training and testing the data is both memory & computational intensive.</p>
<p>Given the smaller configurations of our laptop, this took too much time - we ran into memory errors often. AWS instance also ran out of memory multiple times while data cleaning - but was much better than the laptop</p>
<p></p>

<p><b>   On our Laptop : </b></p>
<p>Time to read crimes data = 56.972469000000004 seconds</p>
<p>Time to split the data into train/test = 10.381031000000007 seconds</p>
<p>Total time taken to train & test LogisticRegression model & test =  491.520608 seconds</p>

<br>
<p><b>   On AWS instance (t2.2xLarge machine) : </b></p>
<p>Time to <b>read crimes data</b> = <b>30.829999999999984 seconds</b></p>
<p>Time to <b>split the data into train/test</b> = <b>6.950000000000003 seconds</b></p>
<p>Total time taken to <b>train Logistic Regression model</b> (10 iterations) and test = <b>164.70999999999998 seconds</b></p>



# Bonus Task

# Option 1: Try to build a Neural Network model, train on the same features and report the accuracy.
Report your observations.

<h4>We decided to use the DNNRegressor to train on the same features as above.</h4>

In [16]:
#Using scikit learn's train_test_split
from sklearn.model_selection import train_test_split

X = crime_data_reduced
y = ylabels
# y = Y_flat[:,0]

X_train_nn, X_test_nn, y_train_nn, y_test_nn = train_test_split(X, y, test_size=0.2)

In [17]:
X_train_nn = X_train_nn.values
X_train_nn = pd.DataFrame(X_train_nn)

In [18]:
X_test_nn = X_test_nn.values
X_test_nn = pd.DataFrame(X_test_nn)

In [19]:
X_test_nn.shape

(200000, 1052)

In [20]:
FEATURES = (X_train_nn.columns).values

#we need to convert Feature column names as type string : this is a restriction for DNN Resgressor with Tensors
FEATURES = FEATURES.astype('str')


In [21]:
X_train_nn.columns = X_train_nn.columns.astype(str)
X_test_nn.columns = X_test_nn.columns.astype(str)

In [22]:
import tensorflow as tf

In [23]:
def get_input_fn(X,Y, num_epochs=None, shuffle=True):
    return tf.estimator.inputs.pandas_input_fn(
      #x=pd.DataFrame({k: X[k].values for k in range(len(FEATURES))}),
      x=X,
      y=pd.Series(Y),
      num_epochs=num_epochs,
      shuffle=shuffle)

In [24]:
import itertools

In [25]:
# Feature cols
feature_cols = [tf.feature_column.numeric_column(k) for k in FEATURES]

  # Build 2 layer fully connected DNN with 10, 10 units respectively.
regressor = tf.estimator.DNNRegressor(feature_columns=feature_cols,
                                        hidden_units=[10,10],
                                        model_dir="/tmp/chicago_DNN_model")

# Train
regressor.train(input_fn=get_input_fn(X_train_nn,y_train_nn), steps=10)

# Evaluate loss over one epoch of test_set.
ev = regressor.evaluate(
      input_fn=get_input_fn(X_test_nn,y_test_nn, num_epochs=1, shuffle=False))
loss_score = ev["loss"]
print("Loss: {0:f}".format(loss_score))

  # Print out predictions over a slice of prediction_set.
y = regressor.predict(
      input_fn=get_input_fn(X_test_nn,y_test_nn, num_epochs=1, shuffle=False))
  # .predict() returns an iterator of dicts; convert to a list and print
  # predictions
predictions = list(p["predictions"] for p in itertools.islice(y, 6))
print("Predictions: {}".format(str(predictions)))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/boston_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f5d4f6bd400>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, 

In [30]:
predictions1 = list(p["predictions"] for p in itertools.islice(y, 100))
print("Predictions1: {}".format(str(predictions1)))

Predictions1: [array([374.6837], dtype=float32), array([-270.27994], dtype=float32), array([274.46664], dtype=float32), array([-634.3134], dtype=float32), array([54.9012], dtype=float32), array([429.76767], dtype=float32), array([-387.50845], dtype=float32), array([-759.50165], dtype=float32), array([245.7762], dtype=float32), array([-452.29996], dtype=float32), array([-142.64275], dtype=float32), array([-104.33952], dtype=float32), array([1231.328], dtype=float32), array([338.43808], dtype=float32), array([159.37605], dtype=float32), array([-586.7692], dtype=float32), array([737.3387], dtype=float32), array([-409.45596], dtype=float32), array([423.07285], dtype=float32), array([-310.0497], dtype=float32), array([-320.99234], dtype=float32), array([-12.8553915], dtype=float32), array([39.281082], dtype=float32), array([-267.0961], dtype=float32), array([271.47714], dtype=float32), array([-169.35735], dtype=float32), array([258.37534], dtype=float32), array([-237.77507], dtype=float32),

# Option 2: Try to predict other things like Arrest or Description of Crime. 

You can change the features in this task. Report your observations.